# Initializing the SysML v2 API

In [1]:
from __future__ import print_function

import time
import requests
from pprint import pprint
import pandas as pd
import json

#host = "<URL of SysML v2 API provider repository>"
host = "http://sysmlapiserver:9000"

# Get projects

In [2]:
projects_url = f"{host}/projects" 
projects_response = requests.get(projects_url)
parts_tree_project_id = ""

if projects_response.status_code == 200:
    projects = projects_response.json()
    
    df = pd.DataFrame({'Project Name':[], 'Project ID':[]})
    for p in projects:
        df = pd.concat([df, pd.DataFrame({'Project Name':[p['name']], 'Project ID':[p['@id']]})], ignore_index=True)
    display(df)
    
    parts_tree_project = list(filter(lambda p: 'Parts Tree' in p['name'], projects))[0] 
    parts_tree_project_id = parts_tree_project['@id']

,Project Name,Project ID
0,Spacecraft project with branches and tags - 20...,d893a70a-4c18-469e-b4a5-7597424f1829


IndexError: list index out of range

## Query to find all part usages

In [3]:
# Query vehicle1 (PartUsage) in the 
query_input = {
  '@type':'Query',
  'select': ['name','@id','@type','owner'],
  'where': {
    '@type': 'CompositeConstraint',
    'operator': 'and',
    'constraint': [
        {
            '@type': 'PrimitiveConstraint',
            'inverse': False,
            'operator': '=',
            'property': '@type',
            'value': 'PartUsage'
        }
    ]
  }
}

payload = json.dumps(query_input)

vehicle1_id = ''
query_url = f"{host}/projects/{parts_tree_project_id}/query-results" 

query_response = requests.post(query_url, json=query_input)

if query_response.status_code == 200:
    query_response_json = query_response.json()
    
    df = pd.DataFrame({'Part Usage Name':[], 'Part Usage ID':[]})
    for p in query_response_json:
        df = pd.concat([df, pd.DataFrame({'Part Usage Name':[p['name']], 'Part Usage ID':[p['@id']]})], ignore_index=True)
    display(df)

## Query to find Part Usage with name vehicle1

In [4]:
# Query vehicle1 (PartUsage) in the 
data = {
  '@type':'Query',
  'select': ['name','@id','@type','owner'],
  'where': {
    '@type': 'CompositeConstraint',
    'operator': 'and',
    'constraint': [
        {
            '@type': 'PrimitiveConstraint',
            'inverse': False,
            'operator': '=',
            'property': 'name',
            'value': 'vehicle1'
        },
        {
            '@type': 'PrimitiveConstraint',
            'inverse': False,
            'operator': '=',
            'property': '@type',
            'value': 'PartUsage'
        }
    ]
  }
}

payload = json.dumps(data)

vehicle1_id = ''
query_url = f"{host}/projects/{parts_tree_project_id}/query-results" 

query_response = requests.post(query_url, json=data)

if query_response.status_code == 200:
    query_response_json = query_response.json()
    pprint(query_response_json)
    vehicle1_id = query_response_json[0]['@id']